In [ ]:
import sys
import os
from bisect import bisect_left
path_to_AOC_helpers = "/Users/connor/Desktop/Coding Problems/Advent_of_code/2024"
# Add the parent directory to the sys.path
sys.path.append(os.path.abspath(path_to_AOC_helpers))
from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet

from copy import deepcopy
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt
from AOC_helpers import *
sys.setrecursionlimit(10000)

input_file = 'input.txt'
test_file = 'test.txt'

with open(input_file, 'r') as file:
    lines = file.readlines()
    edges = [line.strip().split('-') for line in lines]


# build the graph 
nodes = set()
adj_list = defaultdict(set)
for u, v in edges:
    adj_list[u].add(v)
    adj_list[v].add(u)
    nodes.add(u)
    nodes.add(v)
    
def starts_with(s, c):
    """Helper function that tells us if node starts with a given character"""
    return s[0] == c

# Since we looking only for sets that have a node starting with t in them
# we can ignore all the nodes that both don't start with t or connect to a 
# node that starts with t 

good_nodes = set()
for node in nodes: 
    if starts_with(node, 't') or any(starts_with(u, 't') for u in adj_list[node]):
        good_nodes.add(node)

# we can see this drastically reduces the search space 
print(f'We can reduce the number of nodes to look at from {len(nodes)} to {len(good_nodes)}')


def check_triangle(n1, n2, n3):
    """Check if the three nodes form a triangle """
    return n2 in adj_list[n1] and n3 in adj_list[n1] and n2 in adj_list[n3]


# collect all triangles that have a potential historan in them 
triangles = set()
for n1, n2, n3 in combinations(good_nodes, 3):

    if not any(starts_with(node, 't') for node in [n1, n2, n3]):
        continue
    if check_triangle(n1, n2, n3):
        rep = tuple(sorted([n1, n2, n3]))
        triangles.add(rep)

total_triangles = len(triangles)
print(f'There are {total_triangles} triangles that have a potential historian.')
        

    

We can reduce the number of nodes to look at from 520 to 207
There are 1163 triangles that have a potential historian


In [59]:
# Now we need to look for the largest clique in the graph. One naive way to do this
# is to get a set of all triangles in the graph, check what max sized set of nodes
# they all connect to  and then check if this set of nodes forms a clique. 

def check_clique(nodes, adj_list):
    for node in nodes:
        cur_neighbors = adj_list[node]
        if len(cur_neighbors) < len(nodes) - 1:
            return False
        for u in nodes:
            if u == node:
                continue
            if u not in cur_neighbors:
                return False
    return True 

def get_shared_neighbors(u, v, w):
    shared_neighbors = adj_list[u] & adj_list[v] & adj_list[w]
    return shared_neighbors    
            


In [64]:
# Now get all triangles in the graph not just the historian ones.
        
# only 207 nodes start with t or connect to someone that starts with t. 
triangles = set()
for n1, n2, n3 in combinations(nodes, 3):
    if check_triangle(n1, n2, n3):
        rep = tuple(sorted([n1, n2, n3]))
        triangles.add(rep)
total_triangles = len(triangles)
print(f'There are {total_triangles} triangles in the overall graph')
        

There are 11011 triangles in the overall graph


In [66]:
# "build-out" each triangle and then check if it forms a clique
best = None
max_len = 0

for u, v, w in triangles:
    shared  = get_shared_neighbors(u, v, w)
    total = shared | {u, v, w}
    if len(total) <= max_len:
        continue
    if check_clique(total, adj_list):
        size = len(total)
        if size > max_len:
            best = total
            max_len = size

ans = list(best)
ans.sort()
print(f'The largest clique we found was of size {max_len}. It is comprised of:')
print(','.join(ans))

The largest clique we found was of size 13. It is comprised of:
bm,bo,ee,fo,gt,hv,jv,kd,md,mu,nm,wx,xh
